## Importacion de Librerías

In [3]:
import os
import os.path
import json
import numpy as np
import pandas as pd
import datetime as dt
import ast
from ast import literal_eval

print('Importancion completa')

Importancion completa


# **Ingesta de Datos**

Las siguientes líneas de código nos permitirán acceder a los archivos que se encuentran en la carpeta Dataset y detectar ciertos parámetros necesarios para que la librería pandas pueda leerlos sin ningún inconveniente.

In [4]:
# Conocer que extensiones se encuentran en la carpeta Dataset
dataset_dir = '/content/drive/MyDrive/Colab Notebooks/PI_MLOps/Dataset'
files = os.listdir(dataset_dir)

for archivo in files:
    nombre, extension = os.path.splitext(archivo)
    print("Extensión '{}'".format(
  extension))

Extensión '.csv'
Extensión '.csv'


## Archivos .csv

In [5]:
# Creando una lista con el path de los archivos detectados en la carpeta files
csv_files = []

for file in files:
    if os.path.isfile(os.path.join(dataset_dir, file)) and file.endswith('.csv'):
        csv_files.append(f'{dataset_dir}/{file}')

print(f'csv_files: {csv_files}')

csv_files: ['/content/drive/MyDrive/Colab Notebooks/PI_MLOps/Dataset/credits.csv', '/content/drive/MyDrive/Colab Notebooks/PI_MLOps/Dataset/movies_dataset.csv']


In [90]:
credits_csv = pd.read_csv(csv_files[0])
movies_csv = pd.read_csv(csv_files[1], low_memory = False)  # As 10 columns have mixed types, set low_memory as parameter.

print('archivos .csv leidos correctamente')

archivos .csv leidos correctamente


# **Tratamiento de Datos**

Acontinuación se transformará y limpiará la data de cada archivo .csv por separado.

## **credits_csv**

In [91]:
# Las cinco primeras filas del df
credits_csv.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [92]:
# Las ultimas cinco filas del df
credits_csv.tail()

,cast,crew,id
45471,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...",439050
45472,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",111109
45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",67758
45474,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",227506
45475,[],"[{'credit_id': '593e676c92514105b702e68e', 'de...",461257


In [93]:
# Comprobación del número de columnas, filas, tipos de datos y valores faltantes
credits_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


In [94]:
# Cantidad de registros y columnas respectivamente
entries_original = credits_csv.shape[0]
credits_csv.shape

(45476, 3)

### Valores faltantes
¿cuántos datos faltantes hay y que % representan en base al total de datos?

In [95]:
# Cantidad de valores faltantes por columna
missing_values_count = credits_csv.isnull().sum()
columns = credits_csv.shape[1]
missing_values_count[0:columns]

cast    0
crew    0
id      0
dtype: int64

### Examinando la estructura de los datos por atributo

In [96]:
credits_csv.loc[0]

cast    [{'cast_id': 14, 'character': 'Woody (voice)',...
crew    [{'credit_id': '52fe4284c3a36847f8024f49', 'de...
id                                                    862
Name: 0, dtype: object

In [97]:
credits_csv.crew.loc[390]

"[{'credit_id': '535e69b80e0a264fdb006416', 'department': 'Directing', 'gender': 2, 'id': 126837, 'job': 'Director', 'name': 'Michael A. Nickles', 'profile_path': None}, {'credit_id': '535e69c60e0a264fe80065ef', 'department': 'Writing', 'gender': 2, 'id': 126837, 'job': 'Writer', 'name': 'Michael A. Nickles', 'profile_path': None}]"

In [98]:
credits_csv.cast.loc[6402]

"[{'cast_id': 1, 'character': 'Puck', 'credit_id': '52fe43e6c3a368484e004da9', 'gender': 2, 'id': 54455, 'name': 'Devon Gummersall', 'order': 0, 'profile_path': '/9mw9fqm1dtxObQbG6lUGvlvXvt7.jpg'}, {'cast_id': 2, 'character': 'Johnny Black', 'credit_id': '52fe43e6c3a368484e004dad', 'gender': 2, 'id': 52476, 'name': 'Dylan Bruno', 'order': 1, 'profile_path': '/4JdgfAHPodXaJEHNeogxboQtMZp.jpg'}, {'cast_id': 3, 'character': 'Karla', 'credit_id': '52fe43e6c3a368484e004db1', 'gender': 1, 'id': 56356, 'name': 'Gina Philips', 'order': 2, 'profile_path': '/pj9O7yhpZ9JA0fP4bFu3l1xFKhl.jpg'}, {'cast_id': 4, 'character': 'Sweeney', 'credit_id': '52fe43e7c3a368484e004db5', 'gender': 2, 'id': 73589, 'name': 'Johnny Whitworth', 'order': 3, 'profile_path': '/xkeywGnloZXCsHjKz6eqbKoClrh.jpg'}, {'cast_id': 8, 'character': 'Jody', 'credit_id': '52fe43e7c3a368484e004dcb', 'gender': 0, 'id': 15012, 'name': 'Katharine Towne', 'order': 4, 'profile_path': '/33AK0lDtxIhh7125JC7daDWMSkC.jpg'}, {'cast_id': 10, 

### Buscando inconsistencias en la entrada de los datos (Data Entry)

#### Atributo id

In [99]:
unique_id = credits_csv.id.unique()
unique_id.sort()
print(unique_id)

[     2      3      5 ... 468343 468707 469172]


In [100]:
credits_csv.id.value_counts().head()

141971    3
298721    2
9755      2
10991     2
99080     2
Name: id, dtype: int64

Hay id duplicados. Antes de eliminarnos se procedera primero a desanidar la data, extraer la informacion necesaria y finalmente examinar estos registros duplicados.

#### Atributos cast y crew

In [101]:
filtro_cast = credits_csv[credits_csv['cast'].str.contains('[]', regex = False)]
filtro_cast.head()

,cast,crew,id
137,[],"[{'credit_id': '52fe4ab0c3a368484e161d3d', 'de...",124639
240,[],"[{'credit_id': '52fe464ac3a36847f80f6d61', 'de...",43475
393,[],"[{'credit_id': '52fe4624c3a36847f80ef0a5', 'de...",42981
438,[],"[{'credit_id': '52fe448dc3a368484e029383', 'de...",24257
595,[],"[{'credit_id': '52fe4aacc3a368484e16115b', 'de...",124472


In [102]:
empty_data_cast = filtro_cast.cast.shape[0]
print(f'Hay {empty_data_cast} registros en la columna cast con strings que contienen una lista vacia "[]"')

Hay 2418 registros en la columna cast con strings que contienen una lista vacia "[]"


In [103]:
filtro_crew = credits_csv[credits_csv['crew'].str.contains('[]', regex = False)]
filtro_crew.head()

,cast,crew,id
189,"[{'cast_id': 4, 'character': 'Himself', 'credi...",[],56088
614,"[{'cast_id': 1, 'character': 'Grace Rhodes', '...",[],123505
635,"[{'cast_id': 0, 'character': 'Joachim Krippo',...",[],339428
661,[],[],318177
711,[],[],365371


In [104]:
empty_data_crew = filtro_crew.crew.shape[0]
print(f'Hay {empty_data_crew} registros en la columna crew con strings que contienen una lista vacia "[]"')

Hay 771 registros en la columna crew con strings que contienen una lista vacia "[]"


**Observacion:**

Hay que tomar en cuenta que todos estos datos ```'[]'``` representan valores NaN, por lo que las variables ```empty_data_cast``` y ```empty_data_crew``` se utilizaran mas adelante para comparar los valores NaN resultantes de la data limpia.

### Desaninando la Data

Como los datos se presentan en forma de listas "stringificadas" ("stringified" lists) se procede a tratarlos, con el objetivo de que obtengan una estructura manejable para asi poder extraer la información que necesitamos.

In [105]:
# Trabajaremos sobre una copia para no afectar los datos originales
copy_credits = credits_csv.copy()

In [106]:
# Utilizando la función literal_eval para convertir una cadena que contiene una lista en una lista de objetos
columns_credits = ['cast', 'crew']

for atribute in columns_credits:
  copy_credits[atribute] = copy_credits[atribute].apply(lambda x: ast.literal_eval(str(x)))

copy_credits.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': 'Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [107]:
# Las key-value que se encuentran en el atributo cast
copy_credits.cast.loc[0][0]

{'cast_id': 14,
 'character': 'Woody (voice)',
 'credit_id': '52fe4284c3a36847f8024f95',
 'gender': 2,
 'id': 31,
 'name': 'Tom Hanks',
 'order': 0,
 'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'}

In [108]:
# Las key-value que se encuentran en el atributo crew
copy_credits.crew.loc[0][0]

{'credit_id': '52fe4284c3a36847f8024f49',
 'department': 'Directing',
 'gender': 2,
 'id': 7879,
 'job': 'Director',
 'name': 'John Lasseter',
 'profile_path': '/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg'}

In [109]:
pd.DataFrame(copy_credits.crew[0])['job'].value_counts()

Animation                         26
Character Designer                 8
Visual Effects                     5
Assistant Editor                   4
Screenplay                         4
Assistant Sound Editor             4
Original Story                     4
Set Dresser                        3
Sculptor                           2
Production Coordinator             2
Negative Cutter                    2
Layout                             2
Sound Re-Recording Mixer           2
Animation Director                 2
Lighting Supervisor                2
CG Painter                         2
Orchestrator                       2
Editor                             2
Executive Producer                 2
Producer                           2
Supervising Sound Editor           1
Sound Effects Editor               1
Unit Publicist                     1
Sound Design Assistant             1
Editorial Coordinator              1
Casting Consultant                 1
Editorial Manager                  1
D

Del atributo cast se extraerán los nombres de los actores de las peliculas, mientras que para el caso del atributo crew se extraerá el nombre de los directores de cada película.

#### Columna crew

In [110]:
# Funcion para extrer el nombre del director en el atributo crew
def get_director(x):
  lst = []
  if isinstance(x, list) == True:
    if len(x) == 0:
      return np.nan
    for i in x:
      if i['job'] == 'Director':
        lst.append(i['name'])
    if len(lst) == 0:
      return np.nan
    return ','.join(lst)
  else:
    return np.nan

print('Función creada')

Función creada


In [111]:
copy_credits['director'] = copy_credits['crew'].apply(lambda x: get_director(x))
copy_credits.head()

,cast,crew,id,director
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,John Lasseter
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844,Joe Johnston
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602,Howard Deutch
3,"[{'cast_id': 1, 'character': 'Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357,Forest Whitaker
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862,Charles Shyer


In [112]:
# Verificando si hay stringd vacios
copy_credits[copy_credits['director'] == '']

,cast,crew,id,director


In [113]:
copy_credits['director'].value_counts(dropna = False).head(10)

NaN                 887
John Ford            63
Michael Curtiz       61
Alfred Hitchcock     52
Werner Herzog        52
Georges Méliès       51
Woody Allen          47
Sidney Lumet         45
Charlie Chaplin      43
Henry Hathaway       41
Name: director, dtype: int64

#### Columna cast

In [114]:
# Función que returna una lista con el top 3 de los actores o la lista entera del atributo cast
def get_combines(x):
  lst = []
  if isinstance(x, list) == True:
    if len(x) == 0:
      return np.nan
    for i in x:
      lst.append(i['name'])
    if len(lst) == 0:
      return np.nan
    return ','.join(lst)
  else:
    return np.nan

print('Función creada')

Función creada


In [115]:
copy_credits['actors'] = copy_credits['cast'].apply(lambda x: get_combines(x))
copy_credits.head()

,cast,crew,id,director,actors
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862,John Lasseter,"Tom Hanks,Tim Allen,Don Rickles,Jim Varney,Wal..."
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844,Joe Johnston,"Robin Williams,Jonathan Hyde,Kirsten Dunst,Bra..."
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602,Howard Deutch,"Walter Matthau,Jack Lemmon,Ann-Margret,Sophia ..."
3,"[{'cast_id': 1, 'character': 'Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357,Forest Whitaker,"Whitney Houston,Angela Bassett,Loretta Devine,..."
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862,Charles Shyer,"Steve Martin,Diane Keaton,Martin Short,Kimberl..."


In [116]:
# Verificando si hay un string vacio
copy_credits[copy_credits['actors'] == '']

,cast,crew,id,director,actors


In [117]:
copy_credits['actors'].value_counts(dropna = False).head()

NaN               2418
Georges Méliès      24
Louis Theroux       15
Mel Blanc           12
Jimmy Carr           9
Name: actors, dtype: int64

Los atributos que seran utilizados posteriormente seran: id, director y actors

In [118]:
credits = copy_credits[['id', 'director', 'actors']]
credits.head()

,id,director,actors
0,862,John Lasseter,"Tom Hanks,Tim Allen,Don Rickles,Jim Varney,Wal..."
1,8844,Joe Johnston,"Robin Williams,Jonathan Hyde,Kirsten Dunst,Bra..."
2,15602,Howard Deutch,"Walter Matthau,Jack Lemmon,Ann-Margret,Sophia ..."
3,31357,Forest Whitaker,"Whitney Houston,Angela Bassett,Loretta Devine,..."
4,11862,Charles Shyer,"Steve Martin,Diane Keaton,Martin Short,Kimberl..."


#### Valores Faltantes en la data limpia

Verificando valores nulos y comparando con los datos contenidos en las variables ```empty_data_cast``` y ```empty_data_crew```

In [119]:
# Cantidad de valores faltantes por columna
missing_values_count = credits.isnull().sum()
columns = credits.shape[1]
missing_values_count[0:columns]

id             0
director     887
actors      2418
dtype: int64

In [120]:
# Comparativa de los NaN con '[]' en crew
data_nan_director = credits['director'].isnull().sum()
print(f'Cantidad de registros del tipo string con listas vacias {empty_data_crew} en crew')
print(f'Cantidad de registros NaN: {data_nan_director} en crew')

Cantidad de registros del tipo string con listas vacias 771 en crew
Cantidad de registros NaN: 887 en crew


Los 887 datos NaN se deben a la cantidad original de listas vacias sumado a que las listas no vacias 116 de ellas no tenian en los diccionarios un valor de 'Director' para la clave 'Job'

In [123]:
# Comparativa de los NaN con '[]' en actors
data_nan_actors = credits['actors'].isnull().sum()
print(f'Cantidad de registros del tipo string con listas vacias {empty_data_cast} en cast')
print(f'Cantidad de registros NaN: {data_nan_actors} en actors')

Cantidad de registros del tipo string con listas vacias 2418 en cast
Cantidad de registros NaN: 2418 en actors


La cantidad de listas vacias coincide con la cantidad de datos NaN, lo que indica que en todas las listas no vacias se encontro el valor correspondiente para los nombres de los actores.

**Observacion:**

Por ahora, no se tomara ninguna decision con respecto a los datos nulos dado que a continuacion se procede a la limpieza de los datos en el dataframe movies_csv de manera de poder realizar al final un merge() entre los dos dataframes (credicts con movies) y a partir de ahi se decidira que accion tomar referente a la data faltante.

### Comprobando y removiendo datos duplicados

In [124]:
# Verificando si hay datos duplicados en la columna id
credits[credits.duplicated(subset = 'id', keep = False)].shape[0]  # shape[0] numero de filas

87

In [125]:
# Los 5 id que mas se repiten
credits['id'].value_counts().head()

141971    3
298721    2
9755      2
10991     2
99080     2
Name: id, dtype: int64

In [126]:
# Verificando las 3 apariciones del id = 141971
credits[credits['id'] == 141971]

,id,director,actors
13261,141971,JP Siili,"Petteri Summanen,Ismo Kallio,Eppu Salminen,Iri..."
13375,141971,JP Siili,"Petteri Summanen,Ismo Kallio,Eppu Salminen,Iri..."
16764,141971,JP Siili,"Petteri Summanen,Ismo Kallio,Eppu Salminen,Iri..."


Hay 87 id duplicados, los cuales se procede a eliminar

In [132]:
# Eliminando los id duplicados
credits_clean = credits.copy()
credits_clean.drop_duplicates(subset = 'id', inplace = True)

In [133]:
# Comprobando que ya no ha id duplicados
credits_clean[credits_clean.duplicated(subset = 'id', keep = False)].shape[0]

0

In [135]:
# Verificando que el id = 141971 ya no se repite 3 veces
credits_clean[credits_clean['id'] == 141971]

,id,director,actors
13261,141971,JP Siili,"Petteri Summanen,Ismo Kallio,Eppu Salminen,Iri..."


In [138]:
print(f'Cantidad de registros con duplicados: {entries_original}')
entries_final = credits_clean.shape[0]
print(f'Cantidad de registros sin duplicados: {entries_final}')
print(f'% de datos eliminados respecto a la cantidad orginal: {100-(entries_final*100/entries_original)}')

Cantidad de registros con duplicados: 45476
Cantidad de registros sin duplicados: 45432
% de datos eliminados respecto a la cantidad orginal: 0.09675433195531014


Finalmente:

In [139]:
credits_clean.head()

,id,director,actors
0,862,John Lasseter,"Tom Hanks,Tim Allen,Don Rickles,Jim Varney,Wal..."
1,8844,Joe Johnston,"Robin Williams,Jonathan Hyde,Kirsten Dunst,Bra..."
2,15602,Howard Deutch,"Walter Matthau,Jack Lemmon,Ann-Margret,Sophia ..."
3,31357,Forest Whitaker,"Whitney Houston,Angela Bassett,Loretta Devine,..."
4,11862,Charles Shyer,"Steve Martin,Diane Keaton,Martin Short,Kimberl..."


## **movies_csv**

In [140]:
# Las cinco primeras filas del df
movies_csv.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [141]:
# Las últimas cinco filas del df
movies_csv.tail()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0
45465,False,NaN,0,[],NaN,461257,tt6980792,en,Queerama,50 years after decriminalisation of homosexual...,...,2017-06-09,0.0,75.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Queerama,False,0.0,0.0


In [142]:
# Comprobación del número de columnas, filas, tipos de datos y valores faltantes
movies_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [143]:
# Cantidad de filas y columnas respectivamente
entries_original = movies_csv.shape[0]
movies_csv.shape

(45466, 24)

### Datos faltantes

¿cuántos datos faltantes hay y que % representan en base al total de datos?

In [144]:
# Cantidad de datos faltantes por columna
missing_values_count = movies_csv.isnull().sum()
columns = movies_csv.shape[1]
missing_values_count[0:columns]

adult                        0
belongs_to_collection    40972
budget                       0
genres                       0
homepage                 37684
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   5
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25054
title                        6
video                        6
vote_average                 6
vote_count                   6
dtype: int64

In [145]:
# Porcentaje de datos faltantes respecto al total de registros
total_cells = np.product(movies_csv.shape)
total_missing = missing_values_count.sum()
percent_missing = (total_missing / total_cells) * 100
percent_missing

9.674078798809367

### Eliminando atributos

Dado que las columnas ```adult```, ```imdb_id```, ```homepage```, ```original_title```, ```poster_path```, ```video``` no serán utilizadas, se procede a eliminarlas.

In [146]:
columns_drop = ['adult', 'imdb_id', 'homepage', 'original_title', 'poster_path', 'video']
movies_csv.drop(columns_drop, axis = 1, inplace = True)
movies_csv.columns

Index(['belongs_to_collection', 'budget', 'genres', 'id', 'original_language',
       'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

### Examinando la data anidada por atributo

In [147]:
movies_csv.loc[0]

belongs_to_collection    {'id': 10194, 'name': 'Toy Story Collection', ...
budget                                                            30000000
genres                   [{'id': 16, 'name': 'Animation'}, {'id': 35, '...
id                                                                     862
original_language                                                       en
overview                 Led by Woody, Andy's toys live happily in his ...
popularity                                                       21.946943
production_companies        [{'name': 'Pixar Animation Studios', 'id': 3}]
production_countries     [{'iso_3166_1': 'US', 'name': 'United States o...
release_date                                                    1995-10-30
revenue                                                        373554033.0
runtime                                                               81.0
spoken_languages                  [{'iso_639_1': 'en', 'name': 'English'}]
status                   

In [148]:
movies_csv.belongs_to_collection.loc[0]

"{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}"

In [149]:
movies_csv.genres.loc[0]

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

In [150]:
movies_csv.production_companies.loc[0]

"[{'name': 'Pixar Animation Studios', 'id': 3}]"

In [151]:
movies_csv.production_countries.loc[0]

"[{'iso_3166_1': 'US', 'name': 'United States of America'}]"

In [152]:
movies_csv.spoken_languages.loc[0]

"[{'iso_639_1': 'en', 'name': 'English'}]"

### Buscando inconsistencias en los datos anidados

#### Atributo belongs_to_collection

In [174]:
# Verificando si hay registros del tipo strings con diccionarios contenidos vacios
filtro_btc = movies_csv[movies_csv['belongs_to_collection'].str.contains('{}', na = False, regex = False)]
filtro_btc

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count


#### Atributo genres

In [154]:
# Verificando si hay registros del tipo strings con listas contenidas vacias
filtro_genres = movies_csv[movies_csv['genres'].str.contains('[]', regex = False)]
filtro_genres.genres.head()

55     []
83     []
126    []
137    []
390    []
Name: genres, dtype: object

In [155]:
empty_data_genres = filtro_genres.genres.shape[0]
print(f'Hay {empty_data_genres} registros en la columna genres con strings que contienen una lista vacia "[]"')

Hay 2442 registros en la columna genres con strings que contienen una lista vacia "[]"


#### Atributo production_companies

In [157]:
# Verificando si hay registros del tipo strings con listas contenidas vacias
filtro_pc = movies_csv[movies_csv['production_companies'].str.contains('[]', na = False, regex = False)]
filtro_pc.production_companies.head()

50    []
52    []
57    []
58    []
83    []
Name: production_companies, dtype: object

In [158]:
empty_data_pc = filtro_pc['production_companies'].shape[0]
print(f'Hay {empty_data_pc} registros en la columna production_companies con strings que contienen una lista vacia "[]"')

Hay 11875 registros en la columna production_companies con strings que contienen una lista vacia "[]"


#### Atributo production_countries

In [159]:
# Verificando si hay registros del tipo strings con listas contenidas vacias
filtro_countries = movies_csv[movies_csv['production_countries'].str.contains('[]', na = False, regex = False)]
filtro_countries.production_countries.head()

50     []
55     []
83     []
106    []
107    []
Name: production_countries, dtype: object

In [160]:
empty_data_countries = filtro_countries['production_countries'].shape[0]
print(f'Hay {empty_data_countries} registros en la columna production_countries con strings que contienen una lista vacia "[]"')

Hay 6282 registros en la columna production_countries con strings que contienen una lista vacia "[]"


#### Atributo spoken_languages

In [161]:
# Verificando si hay registros del tipo strings con listas contenidas vacias
filtro_languages = movies_csv[movies_csv['spoken_languages'].str.contains('[]', na = False, regex = False)]
filtro_languages.spoken_languages.head()

50     []
83     []
107    []
126    []
137    []
Name: spoken_languages, dtype: object

In [162]:
empty_data_languages = filtro_languages['spoken_languages'].shape[0]
print(f'Hay {empty_data_languages} registros en la columna spoken_languages con strings que contienen una lista vacia "[]"')

Hay 3829 registros en la columna spoken_languages con strings que contienen una lista vacia "[]"


### Desinando la data

Al igual que en el dataset credits_csv, hay atributos donde los datos se presentan en forma de listas "stringificadas" ("stringified" lists) por lo que de la misma forma se procede a tratarlos, con el objetivo de que obtengan una estructura manejable para poder extraer la información que necesitamos.

In [172]:
# Trabajaremos sobre una copia para no afectar los datos originales
copy_movies = movies_csv.copy()

In [173]:
# Funcion para convertir una cadena que contiene una lista o diccionario en una lista o diccionario de objetos respectivamente
def stringified(x):
  if isinstance(x, str) == True:
    lst = ast.literal_eval(x)
    return lst
  else:
    return np.nan

print('Function created')

Function created


In [175]:
columns_movies = ['belongs_to_collection', 'genres', 'production_companies', 'production_countries', 'spoken_languages']

for atribute in columns_movies:
  copy_movies[atribute] = copy_movies[atribute].apply(lambda x: stringified(x))

copy_movies.head(3)

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.7,5415.0
1,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,en,A family wedding reignites the ancient feud be...,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0


Ahora se procede a extraer el valor de la clave 'name' que se encuentra en los atributos belongs_to_collection, genres, production_companies, production_countries, spoken_languages.

#### Atributo belongs_to_collection

In [176]:
# Funcion que recibe un diccionario y retorna un string con el valor de la clave name o NaN
def get_name(x):
  if isinstance(x, dict) == True:
    name = x['name']
    return name
  else:
    return np.nan

print('Functions created')

Functions created


In [177]:
# Usamos la funcion get_name para el caso del atributo belongs_to_collection dado que sus registros son del tipo dict
copy_movies['collection'] = copy_movies['belongs_to_collection'].apply(lambda x: get_name(x))
copy_movies['collection'].head()

0              Toy Story Collection
1                               NaN
2         Grumpy Old Men Collection
3                               NaN
4    Father of the Bride Collection
Name: collection, dtype: object

In [178]:
copy_movies['collection'].value_counts(dropna = False).head()

NaN                              40975
The Bowery Boys                     29
Totò Collection                     27
James Bond Collection               26
Zatôichi: The Blind Swordsman       26
Name: collection, dtype: int64

#### Atributos genres, production_companies, production_countries, spoken_languages

In [170]:
# Funcion que recibe una lista con un diccionario anidado y retorna un string con el valor de la clave name o NaN
def get_combines(x):
  lst = []
  # Evalua si el dato es del tipo lista de no serlo retorna un NaN
  if isinstance(x, list) == True:
    if len(x) == 0:
      return np.nan
    for i in x:
      lst.append(i['name'])
    if len(lst) == 0:
      return np.nan
    return ','.join(lst)
  else:
    return np.nan

'''
def get_combines(x):
    if isinstance(x, list) and len(x) > 0 and all(isinstance(i, dict) and 'name' in i for i in x):
        return ','.join([i['name'] for i in x])
    else:
        return np.nan
'''
print('Functions created')

Functions created


In [179]:
# Usamos la funcion get_combines para el caso de los siguientes atributos dado que sus registros son del tipo lista
columns_movies_1 = ['genres', 'production_companies', 'production_countries', 'spoken_languages']

for atribute in columns_movies_1:
  copy_movies[atribute] = copy_movies[atribute].apply(lambda x: get_combines(x))

In [180]:
# Finalmente confirmamos que ya no hay registros anidados y se extrajo la informacion que solicitamos en las funciones
copy_movies[['collection', 'genres', 'production_companies', 'production_countries', 'spoken_languages']].head()

,collection,genres,production_companies,production_countries,spoken_languages
0,Toy Story Collection,"Animation,Comedy,Family",Pixar Animation Studios,United States of America,English
1,NaN,"Adventure,Fantasy,Family","TriStar Pictures,Teitler Film,Interscope Commu...",United States of America,"English,Français"
2,Grumpy Old Men Collection,"Romance,Comedy","Warner Bros.,Lancaster Gate",United States of America,English
3,NaN,"Comedy,Drama,Romance",Twentieth Century Fox Film Corporation,United States of America,English
4,Father of the Bride Collection,Comedy,"Sandollar Productions,Touchstone Pictures",United States of America,English


In [181]:
copy_movies['genres'].value_counts(dropna = False).head()

Drama            5000
Comedy           3621
Documentary      2723
NaN              2442
Drama,Romance    1301
Name: genres, dtype: int64

In [182]:
empty_data_nan = copy_movies['genres'].isnull().sum()
print(f'Hay {empty_data_nan} datos nulos en la columna genres')
print(f'Habian {empty_data_genres} datos con listas vacias contenidas en un string ("[]")')

Hay 2442 datos nulos en la columna genres
Habian 2442 datos con listas vacias contenidas en un string ("[]")


In [183]:
# Verificando si retornaron strings vacios, debido a que el valor de la clave 'name' fuese ''
copy_movies[copy_movies['genres'] == '']

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,collection


In [184]:
copy_movies['production_companies'].value_counts(dropna = False).head()

NaN                                       11881
Metro-Goldwyn-Mayer (MGM)                   742
Warner Bros.                                540
Paramount Pictures                          505
Twentieth Century Fox Film Corporation      439
Name: production_companies, dtype: int64

In [185]:
empty_data_nan = copy_movies['production_companies'].isnull().sum()
print(f'Hay {empty_data_nan} datos nulos en la columna production_companies')
print(f'Habian {empty_data_pc} datos con listas vacias contenidas en un string ("[]")')

Hay 11881 datos nulos en la columna production_companies
Habian 11875 datos con listas vacias contenidas en un string ("[]")


La cantidad de datos nulos en el atributo production_companies aumento debido a que en las listas no vacias hay diccionarios que no tienen el nombre de la compania de produccion

In [186]:
# Verificando si retornaron strings vacios, debido a que el valor de la clave 'name' fuese ''
copy_movies[copy_movies['production_companies'] == '']

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,collection


In [187]:
copy_movies['production_countries'].value_counts(dropna = False).head()

United States of America    17851
NaN                          6288
United Kingdom               2238
France                       1654
Japan                        1356
Name: production_countries, dtype: int64

In [188]:
empty_data_nan = copy_movies['production_countries'].isnull().sum()
print(f'Hay {empty_data_nan} datos nulos en la columna production_countries')
print(f'Habian {empty_data_countries} datos con listas vacias contenidas en un string ("[]")')

Hay 6288 datos nulos en la columna production_countries
Habian 6282 datos con listas vacias contenidas en un string ("[]")


La cantidad de datos nulos en el atributo production_countries aumento debido a que en las listas no vacias hay diccionarios que no tienen el nombre de los paises productores

In [189]:
# Verificando si retornaron strings vacios, debido a que el valor de la clave 'name' fuese ''
copy_movies[copy_movies['production_countries'] == '']

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,collection


In [190]:
copy_movies['spoken_languages'].value_counts(dropna = False)

English                              22395
NaN                                   3835
Français                              1853
日本語                                   1289
Italiano                              1218
                                     ...  
Français,Latin,,Português,English        1
Español,עִבְרִית,English                 1
English,Íslenska,Pусский                 1
فارسی,                                   1
Fulfulde,English                         1
Name: spoken_languages, Length: 1843, dtype: int64

In [191]:
empty_data_nan = copy_movies['spoken_languages'].isnull().sum()
print(f'Hay {empty_data_nan} datos nulos en la columna spoken_languages')
print(f'Habian {empty_data_languages} datos con listas vacias contenidas en un string ("[]")')

Hay 3835 datos nulos en la columna spoken_languages
Habian 3829 datos con listas vacias contenidas en un string ("[]")


La cantidad de datos nulos en el atributo spoken_languages aumento debido a que en las listas no vacias hay diccionarios que no tienen el idioma hablado. Sin embargo, veamos la siguiente linea de codigo:

In [192]:
# Verificando si retornaron strings vacios, debido a que el valor de la clave 'name' fuese ''
copy_movies[copy_movies['spoken_languages'] == ''].spoken_languages

773       
4184      
5283      
7778      
7798      
        ..
42992     
43313     
44597     
44835     
45462     
Name: spoken_languages, Length: 123, dtype: object

In [193]:
movies_csv['spoken_languages'].loc[773]

"[{'iso_639_1': 'tl', 'name': ''}]"

Este problema, se resolvera mas adelante cuando se imputen datos en el dataset

### Corrigiendo el tipo de dato

In [194]:
copy_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   belongs_to_collection  4494 non-null   object 
 1   budget                 45466 non-null  object 
 2   genres                 43024 non-null  object 
 3   id                     45466 non-null  object 
 4   original_language      45455 non-null  object 
 5   overview               44512 non-null  object 
 6   popularity             45461 non-null  object 
 7   production_companies   33585 non-null  object 
 8   production_countries   39178 non-null  object 
 9   release_date           45379 non-null  object 
 10  revenue                45460 non-null  float64
 11  runtime                45203 non-null  float64
 12  spoken_languages       41631 non-null  object 
 13  status                 45379 non-null  object 
 14  tagline                20412 non-null  object 
 15  ti

#### Convertiendo al tipo numérico

El atributo id pandas no pudo leerlo con el formato correcto, por lo que es un indicativo de que la columna posee algun dato no numerico.

Si se utiliza la funcion astype() para cambiar el tipo de dato de id a int, aparece un error ValueError: invalid literal for int() with base 10: '1997-08-20'. Por lo que se procede aplicar un filtro buscando todos los datos que tengan '-'

In [195]:
filtro_id = copy_movies[copy_movies['id'].str.contains('-', regex = False)]
filtro_id

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,collection
19730,0.065736,/ff9qCepilowshEtG2GYWwzt2bs4.jpg,"Carousel Productions,Vision View Entertainment...",1997-08-20,104.0,Released,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29503,1.931659,/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg,"Aniplex,GoHands,BROSTA TV,Mardock Scramble Pro...",2012-09-29,68.0,Released,NaN,NaN,NaN,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35587,2.185485,/zaSf5OG7V8X8gqFvly88zDdRm46.jpg,"Odyssey Media,Pulser Productions,Rogue State,T...",2014-01-01,82.0,Released,Beware Of Frost Bites,NaN,NaN,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Se observa que los registros donde

In [ ]:
copy_movies['id'] = copy_movies['id'].astype('int32', errors = 'raise')

In [ ]:
# La funcion to_numeric de pandas convert argument to a numeric type. The parameter errors = 'coerce' will return NaN if there is an invalid parse.
copy_movies['id'] = pd.to_numeric(copy_movies['id'], errors = 'coerce', downcast = 'integer')
copy_movies['id'].dtypes

dtype('float64')

Pese a que la funcion to_numeric recibio como parametro retornar la data al tipo integer, el id resultante es del tipo float debido a que hay datos nulos.

In [ ]:
# copy_movies['id'] = copy_movies['id'].astype('int32', errors = 'raise')

copy_movies['budget'] = pd.to_numeric(copy_movies['budget'], errors = 'coerce')
copy_movies['popularity'] = pd.to_numeric(copy_movies['popularity'], errors = 'coerce')

#### Parseando la fecha

In [ ]:
# Verificando el formato de la fecha
copy_movies['release_date'].head()

0    1995-10-30
1    1995-12-15
2    1995-12-22
3    1995-12-22
4    1995-02-10
Name: release_date, dtype: object

In [ ]:
# La funcion to_datetime de pandas convert argument to datetime.
# copy_movies['release_date'] = pd.to_datetime(copy_movies['release_date'], format = '%Y-%m-%d')

Al intentar convertir los datos del atributo 'relase_date' a el tipo datetime con la funcion to_datetime de Pandas. Se retorna un ValueError: indicando que 'time data "1" at position 10395 doesn't match format specified'

Por lo que, considerando que el tipo de dato es un object se procede a comprobar la longitud de cada entrada de relase_date para verificar donde se encuentra el error

In [ ]:
# Obtener la longitud de cada entrada
date_lengths = copy_movies.release_date.str.len()

date_lengths.value_counts()

10.0    45376
2.0         2
1.0         1
Name: release_date, dtype: int64

In [ ]:
# Obtener los indices
indices_2 = np.where([date_lengths == 2])[1]
indices_2

array([29503, 35587])

In [ ]:
indices_1 = np.where([date_lengths == 1])[1]
indices_1

array([19730])

In [ ]:
copy_movies.loc[indices_2]

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
29503,NaN,NaN,"Aniplex,GoHands,BROSTA TV,Mardock Scramble Pro...",NaN,68.0,Released,NaN,NaN,NaN,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35587,NaN,NaN,"Odyssey Media,Pulser Productions,Rogue State,T...",NaN,82.0,Released,NaN,NaN,NaN,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
copy_movies.loc[indices_1]

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
19730,NaN,NaN,"Carousel Productions,Vision View Entertainment...",NaN,104.0,Released,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Podemos observar que para los 3 casos hay datos sucios en la columna release_date. Ademas de distintos valores faltantes importantes como el id, el titulo de la pelicula, el estatus y datos incorrectos en el atributo original_language que analizaremos mas adelante.

Por lo tanto, se procede a ejecutar nuevamente la funcion to_datetime de Pandas pero pasando como agurmento errors = 'coerce' que devolvera NaN sobre aquellos valores que generen un error.

In [ ]:
copy_movies['release_date'] = pd.to_datetime(copy_movies['release_date'], format = '%Y-%m-%d', errors = 'coerce')

In [ ]:
copy_movies['release_date'].value_counts(dropna = False)

2008-01-01    136
2009-01-01    121
2007-01-01    118
2005-01-01    111
2006-01-01    101
             ... 
1957-09-26      1
1938-11-21      1
1936-08-19      1
2010-01-27      1
1917-10-21      1
Name: release_date, Length: 17334, dtype: int64

In [ ]:
# Verificando que los tipos de dato sean correctos
copy_movies.dtypes

belongs_to_collection            object
budget                          float64
genres                           object
id                              float64
original_language                object
overview                         object
popularity                      float64
production_companies             object
production_countries             object
release_date             datetime64[ns]
revenue                         float64
runtime                         float64
spoken_languages                 object
status                           object
tagline                          object
title                            object
vote_average                    float64
vote_count                      float64
dtype: object

### Checking and removing duplicate data

In [ ]:
# Verificando la cantiad de duplicados en la columna id
copy_movies[copy_movies.duplicated(subset = 'id', keep = False)].shape[0]

62

In [ ]:
copy_movies['id'].value_counts()

141971.0    3
5511.0      2
132641.0    2
10991.0     2
168538.0    2
           ..
55135.0     1
15877.0     1
72272.0     1
1549.0      1
461257.0    1
Name: id, Length: 45433, dtype: int64

In [ ]:
# Verificando que el id = 141971 es el que aparece mas veces
copy_movies[copy_movies['id'] == 141971]

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
13261,NaN,0.0,"Thriller,Mystery",141971.0,fi,Recovering from a nail gun shot to the head an...,0.411949,Filmiteollisuus Fine,Finland,2008-12-26,0.0,108.0,suomi,Released,Which one is the first to return - memory or t...,Blackout,6.7,3.0
13375,NaN,0.0,"Thriller,Mystery",141971.0,fi,Recovering from a nail gun shot to the head an...,0.411949,Filmiteollisuus Fine,Finland,2008-12-26,0.0,108.0,suomi,Released,Which one is the first to return - memory or t...,Blackout,6.7,3.0
16764,NaN,0.0,"Thriller,Mystery",141971.0,fi,Recovering from a nail gun shot to the head an...,0.411949,Filmiteollisuus Fine,Finland,2008-12-26,0.0,108.0,suomi,Released,Which one is the first to return - memory or t...,Blackout,6.7,3.0


Hay 62 id duplicados, los cuales se procede a eliminar

In [ ]:
# Eliminando los id duplicados
copy_movies.drop_duplicates(subset = 'id', inplace = True)

In [ ]:
# Comprobando que ya no hay id duplicdos
copy_movies[copy_movies.duplicated(subset = 'id', keep = False)].shape[0]

0

In [ ]:
# Verificando ahora que el id = 141971 aparece una vez
copy_movies[copy_movies['id'] == 141971]

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
13261,NaN,0.0,"Thriller,Mystery",141971.0,fi,Recovering from a nail gun shot to the head an...,0.411949,Filmiteollisuus Fine,Finland,2008-12-26,0.0,108.0,suomi,Released,Which one is the first to return - memory or t...,Blackout,6.7,3.0


### Datos faltantes

¿cuántos datos faltantes hay y que % representan en base al total de datos?

In [ ]:
# Cantidad de datos faltantes por columna
missing_values_count = copy_movies.isnull().sum()
columns = copy_movies.shape[1]
missing_values_count[0:columns]

belongs_to_collection    40946
budget                       1
genres                       0
id                           1
original_language           11
overview                   954
popularity                   4
production_companies         4
production_countries         4
release_date                88
revenue                      4
runtime                    261
spoken_languages             4
status                      85
tagline                  25033
title                        4
vote_average                 4
vote_count                   4
dtype: int64

In [ ]:
# percent_missing
total_cells = np.product(copy_movies.shape)
total_missing = missing_values_count.sum()
percent_missing = (total_missing / total_cells) * 100
percent_missing

8.24297026700513

### Imputacion de Datos Faltantes

#### Atributo spoken_languages

Dado que el nombre del idioma hablado se encuentra en un diccionario con su respectivo ISO639-1, se crea una columna iso_639_1 que facilitara la imputacion

In [142]:
copy_movies[copy_movies['spoken_languages'] == ''].spoken_languages.head()

773     
4184    
5283    
7778    
7798    
Name: spoken_languages, dtype: object

In [ ]:
def get_lenguages(x):
    if isinstance(x, list) and len(x) > 0 and all(isinstance(i, dict) and 'iso_639_1' in i for i in x):
        return ','.join([i['name'] for i in x])
    else:
        return np.nan

In [ ]:
copy_movies['iso_639_1'] = copy_movies['spoken_languages'].apply(lambda x: get_lenguages(x))